In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), os.pardir, 'src')) 

In [2]:
#Temporary
import numpy as np
import pandas as pd

In [3]:
from data.extraction import Data

In [4]:
data = Data()

In [5]:
data.extract('raw', 'reto_precios.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   main_name             981 non-null    object 
 1   subtitle              981 non-null    object 
 2   link                  981 non-null    object 
 3   location              981 non-null    object 
 4   price                 981 non-null    object 
 5   attributes            981 non-null    object 
 6   timestamp             981 non-null    object 
 7   id                    981 non-null    int64  
 8   address               702 non-null    object 
 9   since                 981 non-null    object 
 10  description           981 non-null    object 
 11  vendor                981 non-null    object 
 12  lat                   981 non-null    float64
 13  lon                   981 non-null    float64
 14  price_mod             981 non-null    float64
 15  price_currency        9

In [6]:
#def clean_main_name(df=data.raw):
#    for name in df['main_name']:
#        [word for word in df['main_name'] if word not in df['location'].split()]

In [7]:
def remove_duplicated_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes features that have the same data as other features. 
    Input:
    
    Returns:
    New dataframe without duplicated data.
    """
    # Assert features are really duplicated. And run a baseline, showing that is safe to delete the following columns, given that linear regression sets values of close to 0 for them.
    
    df.drop(['id', # PK: here it is not useful.
             'address', # Subset of the feature "location".
             'link', # The link by itself is not useful. Nevertheless, I might consider its use as an external source of information.
             'price_mod', # The problem we are solving is a function of price_mod. If we leave it we can use it to compute price_mod / m2 and get price_square_meter. In this case, it would be useless to use other features.
             #'m2', # The problem we are solving is a function of m2. If we leave it we can use it to compute price_mod / m2 and get price_square_meter. In this case, it would be useless to use other features.
             #'final_price', # The problem we are solving is a function of final_price. If we leave it we can use it to compute price_mod / m2 and get price_square_meter. In this case, it would be useless to use other features.
             'price', # The problem we are solving is a function of price. If we leave it we can use it to compute price_mod / m2 and get price_square_meter. In this case, it would be useless to use other features.
             'attributes', # This feature's data has been extracted, and used to make new features. Such as: m2, num_bedrooms.
             'main_name', # The information here is either trivial or has been used to make new features. Such as: subtitle
             'since', # This feature's data has been extracted, and used to make new features. Such as: since_period, since_value and days_on_site.
             'age_in_years', # Doesn't have information.
             'subtitle', # Constant (no change)
             'price_currency'# Constant (no change)
            ], axis=1, inplace=True)
    
    return df   

  
    """
    Looks like "price" equals "final_price". If true remove "price"
    """
    
    #can_remove_feature = True
    #for price, price_text in list(zip(df['final_price'],df['price'])):
    #    if str(int(price)) not in price_text.split():
    #        print('Can not remove "price" feature, given that not all rows are equivalent to "final_price" feature')
    #        can_remove_feature = False
    #if can_remove_feature: 
    #    df.drop('price', axis=1 ,inplace=True)
    #    print(f'Removed "price" given that "price_final" = "price"')
    
    
    """
    Remove columns that have the same values on all rows
    """
    #if len(df.columns) < 2:
    #    return df
    #for index, feature1 in enumerate(df):
    #    for feature2 in df.columns[index+1:]:
    #        if df[feature1].equals(df[feature2]):
    #            df.drop(feature2, axis=1, inplace=True)
    #            print(f'Removed "{feature2}" given that "{feature1}" = "{feature2}"')
                
    """
    "attributes" appear equal to "num_bedrooms" and "m2". Verify and remove "attributes" if equal.
    """
    #not_found_values_count = 0
    #for index, row in enumerate(df['attributes']):
    #    values = ' '.join(row.split(',')).split()
    #    if str(df['num_bedrooms'][index].astype(int)) not in values and str(df['m2'][index].astype(int)) not in values:
    #        print(f"{str(df['num_bedrooms'][index])} or {str(df['m2'][index])} are not in {values}")
    #        not_found_values_count += 1 
    #if not_found_values_count == 0:
    #    df.drop(['attributes'], axis=1, inplace=True)
        
    """
    "Since" this feature's data has been extracted, and used to make new features. Such as: since_period, since_value and days_on_site.
    """
    #def get_total_days_since_publication(df):
    #    for index, row in enumerate(df['since']):
    #        row = row.replace('días', '1')
    #        row = row.replace('día', '1')
    #        row = row.replace('meses', '30')
    #        row = row.replace('mes', '30')
    #        row = row.replace('años', '365')
    #        row = row.replace('año', '365')

    #        text = row.split()
    #        values = []
    #        for string in text:
    #            if string.isdigit():
    #                values.append(int(string))
    #        df['since'][index] = np.prod(values)
    #    #df['since'].astype('int')
    #    return df


In [8]:
# Find the first dtype different from NoneType in a series
def find_dtype(series):
    for value in series:
        if type(value) != type(None):
            return type(value)
        
def fix_nans(df):
    for feature in df:
        #if find_dtype(series=trxn[str(feature)]) == pd._libs.tslibs.nattype.NaTType:
        #    trxn[str(feature)].fillna('NA', inplace=True)
        if find_dtype(series=df[str(feature)]) == int or find_dtype(series=df[str(feature)]) == float:
            df[str(feature)].fillna(0, inplace=True)
        else:
            df[str(feature)].fillna('NA', inplace=True)
    return df

def fix_monthly_fee(df):
    for index, row in enumerate(df['monthly_fee']):
        text = row.split()
        for string in text:
            if string.isdigit():
                df.loc[index, 'monthly_fee'] = int(string)
                break
            else:
                df.loc[index, 'monthly_fee'] = 0
            #print(text, df['monthly_fee'][index])
    df['monthly_fee'] = df['monthly_fee'].astype('int')
    return df

In [9]:
def get_cat_codes(df, categorical_features):
    df[categorical_features]=df[categorical_features].astype('category')
    
    for feature in categorical_features:
        df[feature] = df[feature].cat.codes
        
    return df


def sort_df_dtype(df):
    
    objects = df[df.select_dtypes(include='object').columns]
    categories = df[df.select_dtypes(include='int8').columns]
    integers = df[df.select_dtypes(include='int').columns]
    floats = df[df.select_dtypes(include='float').columns]
    
    return pd.concat([objects, categories, integers, floats], axis=1)


def normalize_features(df):
    new_df = df.select_dtypes(['int8', 'int16', 'int32', 'int64', 'float64', 'float16', 'float32'])
    
    mean = new_df.mean()
    std = new_df.std()
    new_df = (new_df - mean) / std
    
    return new_df

In [10]:
df.select_dtypes(['int8', 'int16', 'int32', 'int64', 'float64', 'float16', 'float32']).info()

NameError: name 'df' is not defined

# Extract, Transform, Load (ETL)

In [ ]:
# Applied priciples "Dimensionality reduction", "Better perfomance: Optimized size of data", "Converted text classes to categorical codes"

In [ ]:
df = remove_duplicated_features(df=data.raw.copy())

In [ ]:
df = fix_nans(df)

In [ ]:
df = fix_monthly_fee(df)

In [ ]:
categorical_features = ['subtitle',
                            'vendor',
                            'price_currency',
                            'since_period',
                            'disposition',
                            'orientation',
                            'department_type']
df = get_cat_codes(df, categorical_features=categorical_features)

In [ ]:
df = sort_df_dtype(df)

In [ ]:
df = normalize_features(df)

In [ ]:
df.info()

# Exploratory Data Analysis (EDA)

In [ ]:
# Clustering of departments for sale, data distribution, visualizations.

In [ ]:
#Temporary
#import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
features = df.iloc[:,9:]
x_labels = features.columns
features = features.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=features)
_ = ax.set_xticklabels(x_labels, rotation=90)

In [ ]:
# Once the function parameters have been set we can visualize price_square_meter vs some_feature.

# Feature engineering

In [ ]:
#Download external data. Engineer new features. 

### Geocoding web service

    Used to correctly extract the location data. Since we have the (lat, lon) coordinates we can find the address attributes. 
    It is better this way, because the first record in our dataset has an incorrect neighbourhood "Roma Sur". 
    The correct neighbourhood is "Roma Norte" since "Calle Sinaloa 20" is located in the neighbourhood "Roma Norte".

In [ ]:
#Temporary
from geopy.geocoders import Nominatim

In [ ]:
def get_location_attributes(data=data.raw):
    
    mayoralties = get_mayoralties()
    
    location_data = {'road': [], 'neighbourhood': [], 'postcode': [], 'mayoralty': [],
                    'city': [], 'state': [], 'country': []}
    geolocator = Nominatim(user_agent="uteyechea@gmail.com")
    for i in range(len(data)):
        lat = data['lat'][i]
        lon = data['lon'][i]
        location = geolocator.reverse(f"{str(lat)}, {str(lon)}")
        
        try:
            location_data['road'].append(location.raw['address']['road'])
        except:
            location_data['road'].append('NA')
            
        try:    
            location_data['neighbourhood'].append(location.raw['address']['neighbourhood'].replace('Colonia ', ''))
        except:
            location_data['neighbourhood'].append('NA')
            
        try:
            location_data['postcode'].append(location.raw['address']['postcode'])
        except:
            location_data['postcode'].append('NA')
               
        try:
            mayoralty = [string.strip() for string in location.raw['display_name'].split(',') if string.strip() in mayoralties][0]
            location_data['mayoralty'].append(mayoralty)
        except:
            location_data['mayoralty'].append('NA')
            
        try:
            location_data['city'].append(location.raw['address']['city'])
        except:
            location_data['city'].append('NA')
            
        try:
            location_data['state'].append(location.raw['address']['state'])
        except:
            location_data['state'].append('NA')
            
        try:
            location_data['country'].append(location.raw['address']['country'])
        except:
            location_data['country'].append('NA')     
        
    location_data = pd.DataFrame(location_data)
        #if save_to_disk:
        #    location_data = pd.DataFrame(location_data)
        #    location_data.to_csv(os.path.join(data.data_dir, 'external', 'location_data.csv'))
    
    return location_data

def add_location_attributes(df: pd.DataFrame(), location_data: pd.DataFrame): 
    try:
        df.drop('location', axis=1, inplace=True)
    except:
        pass
    new_df = pd.concat([location_data, df], axis=1)
    return new_df

### Índice de Desarrollo Social de la Ciudad de México, 2020 

    Es una medida ponderada que integra las dimensiones de vivienda, 
    acceso a servicios sanitarios (agua, drenaje y excusado), 
    adecuación energética, acceso a internet y disponibilidad de telefonía (fija o celular), 
    así como el rezago educativo, el acceso a los servicios de salud y a la seguridad social.

In [ ]:
def get_idsm():
    indice_desarrollo_social = pd.read_csv(os.path.join(data.data_dir, 'external', 'ids_alcaldias.csv')).fillna('NA')
    return indice_desarrollo_social

def get_mayoralties():
    indice_desarrollo_social = get_idsm()
    mayoralties = indice_desarrollo_social['alcaldia'].unique()
    return mayoralties

In [ ]:
def add_idsm(df):
    desarrollo_social = get_idsm()
    
    ids = {'ids': []}
    for i, mayoralty in enumerate(df['mayoralty']):
        ids['ids'].append(desarrollo_social[desarrollo_social['alcaldia']==mayoralty].iloc[0,3])
    
    ids = pd.DataFrame(ids)
    new_df = pd.concat([ids, df], axis=1)
    return new_df

In [ ]:
geolocator = Nominatim(user_agent="uteyechea@gmail.com") 
lat = 19.409823
lon = -99.162138
location = geolocator.reverse(f"{str(lat)}, {str(lon)}")
location.raw

### Code execution

In [ ]:
location_data = get_location_attributes()
df = add_location_attributes(df, location_data)
df = add_idsm(df)

In [ ]:
df

# Model

In [ ]:
# We will use a linear regresion model. It is a simple model that allows to compute price_per_square as a function of all available features. It will also allow us to see the distribution of weights associated with each feature.

In [ ]:
#Temporary
import tensorflow as tf

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, use_bias=True)
])

### Before external data

In [ ]:
train_df = df.iloc[:, :-1]
target_df = df.iloc[:, -1:]

In [ ]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

history = model.fit(train_df, target_df, epochs=100, verbose=1)

In [ ]:
plt.bar(x = range(len(train_df.columns)),
        height=model.layers[0].kernel[:,0].numpy())
axis = plt.gca()
axis.set_xticks(range(len(train_df.columns)))
_ = axis.set_xticklabels(train_df.columns, rotation=90)

### After external data

In [ ]:
df.info()

In [ ]:
categorical_features = ['ids','road',
                            'neighbourhood',
                            'postcode',
                            'mayoralty',
                            'city',
                            'state']
df = get_cat_codes(df, categorical_features=categorical_features)

In [ ]:
df = sort_df_dtype(df)

In [ ]:
df = normalize_features(df)

In [ ]:
df

In [ ]:
df = df.drop(['subtitle', 'price_currency'], axis=1).info()

### Model

In [ ]:
train_df = df.iloc[:, :-1]
target_df = df.iloc[:, -1:]

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, use_bias=True)
])

In [ ]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

history = model.fit(train_df, target_df, epochs=100, verbose=1)

In [ ]:
plt.bar(x = range(len(train_df.columns)),
        height=model.layers[0].kernel[:,0].numpy())
axis = plt.gca()
axis.set_xticks(range(len(train_df.columns)))
_ = axis.set_xticklabels(train_df.columns, rotation=90)